# Exercise 1

Theoretical Part

# Exercise 2

### 2.2

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# import data using pandas
df = pd.read_csv('housing.csv')
#print(df)

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

train_ratio = 0.60
validation_ratio = 0.20
test_ratio = 0.20

# train is now 60% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=10)

# test is now 20% of the initial data set
# validation is now 20% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=10) 

#print(X_train, X_val, X_test)

In [3]:
# imputation
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='mean')

X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)
X_val = imp.transform(X_val)

In [4]:
# scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

### 2.3

In [5]:
from sklearn.metrics import r2_score
from scipy.optimize import minimize
import numpy as np


class LinReg():
    """
    This class trains linear model with intercept (bias)
    """
    def __init__(self, C = None):
        self.p = None # parameters field of model
        self.C = C

    def predict(self, X):
        if self.p is None:
            raise("Please train the model first!")

        return self._predict(X, self.p)

    def _predict(self, X, p):
        # this is used for optimization of objective
        # returns a vector where i-th elemnt corresponds to
        # the output of the model with i-th input row
        
        return np.dot(X, p[:-1]) + p[-1]
        
        
    def fit(self, X,Y):
        # objective: minimize squared deviation
        def obj(p):
            Yp = self._predict(X, p)
            loss = np.mean((Yp -Y) ** 2)

            # check if regularization is used
            if self.C is None:
                return loss

            return loss +  (self.C * np.sum( np.abs(p) ))


        p0 = np.zeros(X.shape[1] + 1)
        sol = minimize(obj, p0, method="L-BFGS-B", tol=1e-6)
        self.p = sol.x

    def score(self, Xv, Yv):
        # this function gets predictions on Xv and compares to Yv
        # using r2_score function
        Yp = self.predict(Xv)
        return r2_score(Yv, Yp)

In [25]:
# create and train predictive model

regularization_array = [0.01, 1, 100]
for c in regularization_array:
    model = LinReg(C = c)
    model.fit(X_train, y_train)
    print ("--------------")
    print ("Regularization :", c)
    print("The value for r_2_score for regularization is", model.score(X_test, y_test))
    
    # print the model weights
    for c, v in zip( df.columns.values[:-1], model.p[:-1]):
        print(c,v)

--------------
Regularization : 0.01
The value for r_2_score for regularization is 0.7018751805836294
CRIM -1.1405739820871321
ZN 0.6671155233172454
INDUS 0.46749835871801443
CHAS 0.9716896307213567
NOX -2.5178368238127047
RM 3.107347323081306
AGE -0.21894854648546058
DIS -3.0301791557464894
RAD 2.9644437967226644
TAX -2.8977350888998394
PTRATIO -1.7855479908835494
LSTAT -3.0859443375522924
--------------
Regularization : 1
The value for r_2_score for regularization is 0.6768892809290505
CRIM -0.40239261418305444
ZN 0.0001415275654250641
INDUS -0.00031075991857067305
CHAS 0.6488428874148371
NOX -0.8042501693276599
RM 3.2784069658457304
AGE -6.723152447648184e-07
DIS -0.7166262051722561
RAD -0.0007176150009366715
TAX -0.4447806647248969
PTRATIO -1.1379449977303973
LSTAT -2.9098150750448766
--------------
Regularization : 100
The value for r_2_score for regularization is -5.243131547579747
CRIM 0.0
ZN 0.0
INDUS 0.0
CHAS 0.0
NOX 0.0
RM 0.0
AGE 0.0
DIS 0.0
RAD 0.0
TAX 0.0
PTRATIO 0.0
LSTAT